## Pre-processing of IUCN ranges

In [8]:
from tqdm.auto import tqdm

In [9]:
#arcpy.env.workspace = r"C:\Users\olivoshj\Desktop\ArcGIS_Projects\Dam_Project_Aux\Dam_Project_Aux.gdb"
arcpy.env.workspace = r"D:\Andres\Dam_Project_D\Dam_D.gdb"

In [10]:
## Define path to folder containing IUCN ranges for freshwater species 
# Available at: https://www.iucnredlist.org/resources/spatial-data-download
# Link Freshwater Groups\Fishes: https://www.iucnredlist.org/resources/files/424bc197-b56e-4c6c-acca-0c072616d2d9
# version: 2024.2
raw_ranges_shps = r"R:\FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Revised_Analysis_NatureCommunications\Input_datasets\IUCN_Ranges\FW_FISH_2024_2"

In [11]:
# merge shapefiles into a single feature class in ESRI gdb
arcpy.management.Merge(
    inputs=fr"{raw_ranges_shps}\FW_FISH_PART1.shp; {raw_ranges_shps}\FW_FISH_PART2.shp",
    output="FW_FISH_2024_2"
)

<Result 'C:\\Users\\olivoshj\\Desktop\\ArcGIS_Projects\\Dam_Project_Aux\\Dam_Project_Aux.gdb\\FW_FISH_2024_2'>

In [12]:
# filter and separate ranges based on presence and origin fields

arcpy.conversion.ExportFeatures(
    in_features="FW_FISH_2024_2",
    out_features="Extant_Native_Ranges",
    where_clause="presence = 1 And origin = 1"
)

arcpy.conversion.ExportFeatures(
    in_features="FW_FISH_2024_2",
    out_features="Extant_Introduced_Ranges",
    where_clause="presence = 1 And origin = 3"
)

arcpy.conversion.ExportFeatures(
    in_features="FW_FISH_2024_2",
    out_features="Extinct_Native_Ranges",
    where_clause="presence = 5 And origin = 1"
)

# remaining polygons with uncertainty in presence (values 2,3,4) or origin (values 2,5) + single species introduced but extinct (Silurus aristotelis in Lake Volvi, Greece)
arcpy.conversion.ExportFeatures(
    in_features="FW_FISH_2024_2",
    out_features="Dropped_Ranges",
    where_clause="(presence <> 1 And presence <> 5) Or (origin <> 1 And origin <> 3) Or (presence = 5 And origin = 3)"
)

<Result 'C:\\Users\\olivoshj\\Desktop\\ArcGIS_Projects\\Dam_Project_Aux\\Dam_Project_Aux.gdb\\Dropped_Ranges'>

In [13]:
# clip polygons using HydroSHEDS representation of watershed boundaries (drops marine portion of IUCN ranges)

hydro_atlas = "BasinATLAS_v10_lev01"

In [14]:
# clip using lands and dissolve polygons by species

range_types = ["Extant_Introduced_Ranges", "Extinct_Native_Ranges","Extant_Native_Ranges"]

In [15]:
for range_type in range_types:
    # dissolve them to keep one multi-part polygon per species
    arcpy.analysis.PairwiseDissolve(
        in_features=range_type,
        out_feature_class=f"{range_type}_Diss",
        dissolve_field="sci_name;category;class;order_;family;genus"
    )
    
    # clip ranges with lands: can take several hours given the number of polygons
    arcpy.analysis.PairwiseClip(
        in_features=f"{range_type}_Diss",
        clip_features=hydro_atlas,
        out_feature_class=f"{range_type}_Clipped"
    )


In [16]:
# calculate original area and clipped area of each range. Then calculate the marine:freshwater proportion
# assess manually those species with more than 95% area dropped (potential brackish species that need filtering)

arcpy.management.CalculateGeometryAttributes(
    in_features="Extant_Native_Ranges_Diss",
    geometry_property="Original_AreaSqk AREA",
    length_unit="",
    area_unit="SQUARE_KILOMETERS",
    coordinate_system='PROJCS["WGS_1984_Equal_Earth_Greenwich",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Equal_Earth"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]'
)

arcpy.management.CalculateGeometryAttributes(
    in_features="Extant_Native_Ranges_Clipped",
    geometry_property="Clipped_AreaSqk AREA",
    length_unit="",
    area_unit="SQUARE_KILOMETERS",
    coordinate_system='PROJCS["WGS_1984_Equal_Earth_Greenwich",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Equal_Earth"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]'
)

arcpy.management.JoinField(
    in_data="Extant_Native_Ranges_Clipped",
    in_field="sci_name",
    join_table="Extant_Native_Ranges_Diss",
    join_field="sci_name",
    fields="Original_AreaSqk"
)

arcpy.management.AddFields("Extant_Native_Ranges_Clipped", "Freshwater_Percent SHORT")

with arcpy.da.UpdateCursor("Extant_Native_Ranges_Clipped", ['Original_AreaSqk','Clipped_AreaSqk','Freshwater_Percent']) as cursor:
    for row in cursor:
        
        if row[1] is not None:
            row[2] = int(row[1]/row[0]*100)
        else:
            row[2] = 0
            
        cursor.updateRow(row)

In [ ]:
# calculate overlaps with native, introduced and extinct ranges

input_ranges = "Extant_Native_Ranges_Clipped"
fields = ['SHAPE@','Symp_count','Intro_count','Extir_count']

arcpy.management.AddFields(input_ranges, "Symp_count SHORT;Intro_count SHORT;Extir_count SHORT")

with arcpy.da.UpdateCursor(input_ranges,fields) as cursor:
    for row in tqdm(cursor, total=int(arcpy.management.GetCount(input_ranges)[0])):

        if None not in row:
            continue

        for range_type in range_types:
            
            try:
                arcpy.analysis.PairwiseClip(
                    in_features=fr"{range_type}_Diss",
                    clip_features=row[0],
                    out_feature_class="currClip"
                )
    
                overlaps = int(arcpy.management.GetCount("currClip")[0])
    
                if "Extant_Native" in range_type:
                    row[1] = overlaps - 1 # substract one (self overlap)
    
                elif "Extant_Introduced" in range_type:
                    row[2] = overlaps
                    
                elif "Extinct_Native" in range_type:
                    row[3] = overlaps
                    
                cursor.updateRow(row)
                
            except:
                print("Skipped record due to geodatabase lock (arcpy's bug), re-run cell after finishing to fill null columns")
                continue

  4%|▎         | 493/13597 [59:38<24:26:58,  6.72s/it]

Skipped record due to geodatabase lock (arcpy's bug), re-run cell after finishing to fill null columns


  8%|▊         | 1082/13597 [2:15:06<23:52:28,  6.87s/it]

Skipped record due to geodatabase lock (arcpy's bug), re-run cell after finishing to fill null columns


 66%|██████▌   | 8950/13597 [54:46:26<527:33:01, 408.69s/it]  

In [ ]:
# generate a projected (Equal Earth) version for posterior area-sensitive analyses
arcpy.management.Project(
    in_dataset="Extant_Native_Ranges_Clipped",
    out_dataset="Extant_Native_Ranges_EE",
    out_coor_system='PROJCS["WGS_1984_Equal_Earth_Greenwich",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Equal_Earth"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
    in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
)

In [ ]:
arcpy.management.AddFields("Extant_Native_Ranges_EE", "Area_Sqk FLOAT; Perimeter_k FLOAT")

with arcpy.da.UpdateCursor("Extant_Native_Ranges_EE", ['Shape_Area','Shape_Length','Area_Sqk','Perimeter_k']) as cursor:
    for row in cursor:
        row[2] = row[0] / 1000000 # area in square kilometers
        row[3] = row[1] / 1000 # perimeter in kilometers
        cursor.updateRow(row)

In [ ]:
## calculate range-based attributes: north limit, south limit, east limit, west limit

arcpy.management.CalculateGeometryAttributes(
    in_features="Extant_Native_Ranges_EE",
    geometry_property="North_Limit EXTENT_MAX_Y;South_Limit EXTENT_MIN_Y;East_Limit EXTENT_MAX_X;West_Limit EXTENT_MIN_X",
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    coordinate_format="DD"
)

In [ ]:
# save copy as csv file

arcpy.conversion.ExportTable(
    in_table="Extant_Native_Ranges_EE",
    out_table=r"R:\FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Revised_Analysis_NatureCommunications\Input_datasets\Tabular_data_by_species\Ranges_Attributes.csv",
    field_mapping='sci_name "sci_name" true true false 100 Text 0 0,First,#,Extant_Native_Ranges_EE,sci_name,0,99;class "class" true true false 30 Text 0 0,First,#,Extant_Native_Ranges_EE,class,0,29;order_ "order_" true true false 50 Text 0 0,First,#,Extant_Native_Ranges_EE,order_,0,49;family "family" true true false 80 Text 0 0,First,#,Extant_Native_Ranges_EE,family,0,79;genus "genus" true true false 80 Text 0 0,First,#,Extant_Native_Ranges_EE,genus,0,79;category "category" true true false 5 Text 0 0,First,#,Extant_Native_Ranges_EE,category,0,4;Freshwater_Percent "Freshwater_Percent" true true false 2 Short 0 0,First,#,Extant_Native_Ranges_EE,Freshwater_Percent,-1,-1;Symp_count "Symp_count" true true false 2 Short 0 0,First,#,Extant_Native_Ranges_EE,Symp_count,-1,-1;Intro_count "Intro_count" true true false 2 Short 0 0,First,#,Extant_Native_Ranges_EE,Intro_count,-1,-1;Extir_count "Extir_count" true true false 2 Short 0 0,First,#,Extant_Native_Ranges_EE,Extir_count,-1,-1;North_Limit "North_Limit" true true false 8 Double 0 0,First,#,Extant_Native_Ranges_EE,North_Limit,-1,-1;South_Limit "South_Limit" true true false 8 Double 0 0,First,#,Extant_Native_Ranges_EE,South_Limit,-1,-1;East_Limit "East_Limit" true true false 8 Double 0 0,First,#,Extant_Native_Ranges_EE,East_Limit,-1,-1;West_Limit "West_Limit" true true false 8 Double 0 0,First,#,Extant_Native_Ranges_EE,West_Limit,-1,-1;Area_Sqk "Area_Sqk" true true false 4 Float 0 0,First,#,Extant_Native_Ranges_EE,Area_Sqk,-1,-1;Perimeter_k "Perimeter_k" true true false 4 Float 0 0,First,#,Extant_Native_Ranges_EE,Perimeter_k,-1,-1'
)

In [ ]:
# delete intermediate files
arcpy.management.Delete(
    in_data=fr"{arcpy.env.workspace}\currClip;currClip"
)